In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, RANSACRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, train_test_split

# ===========================================================
# Ejemplo de datos simulados (reemplaza con tus datos reales)
# ===========================================================
# X: matriz de features (n_muestras, n_features)
# y: vector o matriz de objetivos (n_muestras,) o (n_muestras, n_targets)
np.random.seed(42)
X = np.random.rand(200, 10)
y = X[:, 0]*3.5 + X[:, 1]*(-2.4) + 0.5 * np.random.randn(200)

# ===========================================================
# Definición de los modelos a probar en un diccionario
# ===========================================================
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'SVR': SVR(),
    'RandomForest': RandomForestRegressor(),
    'KNeighbors': KNeighborsRegressor(),
    'RANSAC': RANSACRegressor()
}

# ===========================================================
# Definición de los hiperparámetros a probar (param_grid)
# Cada clave debe corresponder con la del diccionario 'models'
# ===========================================================
param_grids = {
    'LinearRegression': {
        'fit_intercept': [True, False]
    },
    'Ridge': {
        'alpha': [0.1, 1.0, 10.0, 100.0],
        'fit_intercept': [True, False]
    },
    'SVR': {
        'kernel': ['linear', 'rbf'],
        'C': [0.1, 1, 10, 100],
        'epsilon': [0.01, 0.1, 0.5]
    },
    'RandomForest': {
        'n_estimators': [10, 50, 100],
        'max_depth': [3, 5, 10, None],
        'min_samples_leaf': [1, 2, 5]
    },
    'KNeighbors': {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance']
    },
    'RANSAC': {
        'min_samples': [0.5, 0.75, 1.0],
        'max_trials': [50, 100, 200],
        'stop_n_inliers': [np.inf, 50, 100]  # Algunas opciones adicionales
    }
}

# ===========================================================
# Entrenamiento y búsqueda de hiperparámetros con GridSearchCV
# ===========================================================
best_estimators = {}
best_params = {}
best_scores = {}

# Dividimos en entrenamiento y test para tener un set aparte (opcional)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

for name, model in models.items():
    print(f"\nEntrenando y buscando hiperparámetros para: {name}")
    
    # Definimos el GridSearchCV
    grid = GridSearchCV(
        estimator=model,
        param_grid=param_grids[name],
        cv=5,                # Número de folds para cross-validation
        scoring='r2',        # Métrica para comparar modelos (R^2)
        n_jobs=-1,           # Usa todos los cores disponibles
        verbose=1
    )
    
    # Ajustamos el modelo con la búsqueda de grilla
    grid.fit(X_train, y_train)
    
    # Guardamos los resultados
    best_estimators[name] = grid.best_estimator_
    best_params[name] = grid.best_params_
    best_scores[name] = grid.best_score_
    
    print(f" > Mejor score (cv): {grid.best_score_:.4f}")
    print(f" > Mejor combinación de parámetros: {grid.best_params_}")

# ===========================================================
# Evaluar cada mejor modelo en el set de test (opcional)



Entrenando y buscando hiperparámetros para: LinearRegression
Fitting 5 folds for each of 2 candidates, totalling 10 fits
 > Mejor score (cv): 0.8773
 > Mejor combinación de parámetros: {'fit_intercept': False}

Entrenando y buscando hiperparámetros para: Ridge
Fitting 5 folds for each of 8 candidates, totalling 40 fits
 > Mejor score (cv): 0.8778
 > Mejor combinación de parámetros: {'alpha': 0.1, 'fit_intercept': False}

Entrenando y buscando hiperparámetros para: SVR
Fitting 5 folds for each of 24 candidates, totalling 120 fits
 > Mejor score (cv): 0.8690
 > Mejor combinación de parámetros: {'C': 100, 'epsilon': 0.1, 'kernel': 'linear'}

Entrenando y buscando hiperparámetros para: RandomForest
Fitting 5 folds for each of 36 candidates, totalling 180 fits
 > Mejor score (cv): 0.8481
 > Mejor combinación de parámetros: {'max_depth': 10, 'min_samples_leaf': 5, 'n_estimators': 50}

Entrenando y buscando hiperparámetros para: KNeighbors
Fitting 5 folds for each of 6 candidates, totalling 

In [4]:
# ===========================================================
from sklearn.metrics import r2_score

print("\n======= Resultados en el test set =======")
for name, estimator in best_estimators.items():
    y_pred = estimator.predict(X_test)
    test_score = r2_score(y_test, y_pred)
    print(f"{name} - test R^2: {test_score:.4f}")


======= Resultados en el test set =======
LinearRegression - test R^2: 0.9062
Ridge - test R^2: 0.9061
SVR - test R^2: 0.9001
RandomForest - test R^2: 0.9085
KNeighbors - test R^2: 0.7579
RANSAC - test R^2: 0.9112
